In [1]:
# from google.colab import drive
# import os

# drive.mount('/content/drive')

# drive.mount('/content/drive')

# dataset_path = "/content/drive/My Drive/Uni/Bird Exploration/CUB_200_2011"

# if os.path.exists(dataset_path):
#   print(f"✅ CUB dataset is available at: {dataset_path}")
# else:
#   print("❌ Dataset not found! Run the download script first.")

In [3]:
!pip install ftfy regex tqdm nltk peft
# !pip install git+https://github.com/openai/CLIP.git
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import BitsAndBytesConfig
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader, random_split
# import clip
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os, json
import pandas as pd
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import nltk

from peft import LoraConfig, get_peft_model
from transformers import CLIPProcessor, CLIPModel

nltk.download('punkt')


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\miret\AppData\Local\Temp\pip-req-build-oy0kh6t7'


  Cloning https://github.com/huggingface/transformers.git to c:\users\miret\appdata\local\temp\pip-req-build-oy0kh6t7
  Resolved https://github.com/huggingface/transformers.git to commit 7f5077e53682ca855afc826162b204ebf809f1f9
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\miret\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)
model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

cuda


d:\Users\miret\anaconda3\envs\cvEnv\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\miret\.cache\huggingface\hub\models--openai--clip-vit-base-patch32. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
with open("/data/llava_captions.json", "r") as f:
    dataset = json.load(f)
    
class BirdDataset(Dataset):
    def __init__(self, dataset, root_dir="/kaggle/input/cub2002011/CUB_200_2011/images/"):
        self.data = list(dataset.values())
        self.root_dir = root_dir
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  
            transforms.ToTensor(),  
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        positive_sample = self.data[idx]
        
        # Extract image path and class label
        class_label = positive_sample["class_label"]
        image_filename = os.path.basename(positive_sample["image_path"])
        image_path = os.path.join(self.root_dir, class_label, image_filename)

        # Handle missing images
        if not os.path.exists(image_path):
            print(f"Warning: Missing image {image_path}. Using blank tensor.")
            image = torch.zeros((3, 224, 224))  # Return blank tensor instead of breaking
        else:
            image = Image.open(image_path).convert("RGB")
            image = self.transform(image)

        # Process text: Split into sentences and tokenize
        text = positive_sample.get("llava_text", "").strip()
        if not text:
            text = "No description available."
        sentences = nltk.tokenize.sent_tokenize(text)

        # Convert each sentence into tokenized tensors
        encoded_text = processor(text=sentences, return_tensors="pt", padding=True, truncation=True, max_length=77)

        # Select a negative sample (wrong bird text)
        neg_idx = np.random.randint(0, len(self.data))
        while neg_idx == idx:
            neg_idx = np.random.randint(0, len(self.data))
        negative_text = self.data[neg_idx].get("llava_text", "").strip()
        if not negative_text:
            negative_text = "No negative description available."
        negative_sentences = nltk.tokenize.sent_tokenize(negative_text)

        # Convert negative text into tokenized tensors
        encoded_negative_text = processor(text=negative_sentences, return_tensors="pt", padding=True, truncation=True, max_length=77)

        return image, encoded_text, encoded_negative_text

# **🔹 Custom Collate Function with Dynamic Padding**
def collate_fn(batch):
    images, encoded_texts, encoded_neg_texts = zip(*batch)

    # Convert images to tensor batch
    images = torch.stack(images).to(device)

    # **Find the longest sequence length in the batch**
    max_length = max([et["input_ids"].shape[1] for et in encoded_texts])
    max_neg_length = max([et["input_ids"].shape[1] for et in encoded_neg_texts])

    # **Pad input sequences to max length**
    def pad_sequence(seq_list, max_len):
        batch_size = len(seq_list)
        padded_seq = torch.zeros((batch_size, max_len), dtype=torch.long).to(device)
        attention_masks = torch.zeros((batch_size, max_len), dtype=torch.long).to(device)

        for i, et in enumerate(seq_list):
            length = et["input_ids"].shape[1]
            padded_seq[i, :length] = et["input_ids"].squeeze(0)
            attention_masks[i, :length] = et["attention_mask"].squeeze(0)

        return {"input_ids": padded_seq, "attention_mask": attention_masks}

    # Pad positive and negative texts
    padded_texts = pad_sequence(encoded_texts, max_length)
    padded_neg_texts = pad_sequence(encoded_neg_texts, max_neg_length)

    return images, padded_texts, padded_neg_texts

FileNotFoundError: [Errno 2] No such file or directory: '/data/llava_captions.json'

In [4]:
train_dataset = BirdDataset(dataset)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)


optimizer = torch.optim.Adam(clip_model.parameters(), lr=5e-5)
loss_fn = torch.nn.CosineEmbeddingLoss()  # Contrastive loss



In [6]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["visual_projection", "text_projection"],
    lora_dropout=0.1
)
clip_model = get_peft_model(clip_model, config)

In [7]:
train_dataset = BirdDataset(dataset)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2, collate_fn=collate_fn)

# Define Loss Function and Optimizer
optimizer = torch.optim.AdamW(clip_model.parameters(), lr=5e-5)
loss_fn = nn.CosineEmbeddingLoss()

# Training Loop
epochs = 5
for epoch in range(epochs):
    clip_model.train()
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=True)

    total_loss = 0

    for images, encoded_texts, encoded_neg_texts in progress_bar:
        images = images.to(device)

        # Convert tokenized text to device
        input_ids = encoded_texts["input_ids"].to(device)
        attention_mask = encoded_texts["attention_mask"].to(device)

        neg_input_ids = encoded_neg_texts["input_ids"].to(device)
        neg_attention_mask = encoded_neg_texts["attention_mask"].to(device)

        # Process text embeddings
        text_embeddings = clip_model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
        neg_text_embeddings = clip_model.get_text_features(input_ids=neg_input_ids, attention_mask=neg_attention_mask)

        # Compute similarity loss
        labels = torch.ones(text_embeddings.shape[0]).to(device)  # Positive samples
        neg_labels = -torch.ones(neg_text_embeddings.shape[0]).to(device)  # Negative samples

        loss = loss_fn(text_embeddings, neg_text_embeddings, labels) + loss_fn(neg_text_embeddings, text_embeddings, neg_labels)
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update progress bar
        progress_bar.set_postfix(loss=total_loss / (progress_bar.n + 1))

    print(f"Epoch {epoch+1} Loss: {total_loss / len(train_loader)}")

# Save Model
torch.save(clip_model.state_dict(), "clip_finetuned_lora.pth")
print("✅ Training complete. Model saved as clip_finetuned_lora.pth")

Epoch 1/5:   0%|          | 0/2947 [00:00<?, ?it/s]


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
  File "<ipython-input-4-3350f20a207d>", line 60, in collate_fn
    images = torch.stack(images).to(device)
  File "/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py", line 305, in _lazy_init
    raise RuntimeError(
RuntimeError: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method


In [ ]:
def retrieve_image(query_text):
    inputs = processor(text=[query_text], return_tensors="pt").to(device)

    with torch.no_grad():
        text_features = clip_model(**inputs).text_embeds

    similarities = {}

    for key, sample in dataset.items():
        image = Image.open(sample["image_path"]).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            image_features = clip_model(**inputs).image_embeds

        similarity = torch.nn.functional.cosine_similarity(text_features, image_features)
        similarities[key] = similarity.item()

    best_match = max(similarities, key=similarities.get)
    return dataset[best_match]["image_path"]

# Example test
query_text = "A cliff swallow with a red forehead perched on a wooden post."
print(retrieve_image(query_text))


In [ ]:
def retrieve_text(image_path):
    image = Image.open(image_path).convert("RGB")
    image_inputs = processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        image_features = clip_model.get_image_features(**image_inputs)

    similarities = {}

    for key, sample in dataset.items():
        text = sample["llava_text"]
        text_inputs = processor(text=[text], return_tensors="pt", padding=True, truncation=True).to(device)

        with torch.no_grad():
            text_features = clip_model.get_text_features(**text_inputs)

        similarity = torch.nn.functional.cosine_similarity(image_features, text_features)
        similarities[key] = similarity.item()

    best_match = max(similarities, key=similarities.get)
    return dataset[best_match]["llava_text"]

test_image = "/kaggle/input/cub2002011/CUB_200_2011/images/002.Laysan_Albatross/Laysan_Albatross_0001_545.jpg"
print(retrieve_text(test_image))